# Agregando festures

In [1]:
import os
from os.path import join, dirname
from dotenv import load_dotenv

dotenv_path = join(dirname('__file__'), '.env')

load_dotenv(dotenv_path)

ROOT_PATH = os.environ.get("ROOT_PATH")
DATALAKE_PATH = os.environ.get("DATALAKE_PATH")


In [6]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from glob import glob


In [10]:
files = glob(f"{DATALAKE_PATH}/siena/processed/features/*.mat")


In [46]:
def pipeline(file):

    d = loadmat(file)
    labels = ['ictal', 'pre', 'pos', 'normal', 'rep']
    dfs = []

    for label in labels:
        use_keys = {item for item in d.keys() if label == item.split('_')[1]}
        extracted_dict = {key: value for key,
                          value in d.items() if key in use_keys}

        for i in extracted_dict.keys():
            extracted_dict[i] = extracted_dict[i][0]

        new_df = pd.DataFrame(extracted_dict)

        new_df.columns = ['var', 'skew', 'kur']

        new_df['label'] = np.repeat(label, len(new_df))

        dfs.append(new_df)

    return pd.concat(dfs)


In [47]:
D = []

for file in files:
    D.append(pipeline(file))

In [50]:
df = pd.concat(D)

In [51]:
df.to_parquet(f"{ROOT_PATH}/features/features.parquet")